In [1]:
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from matplotlib.colors import TwoSlopeNorm

import mne
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
from mne.stats import permutation_cluster_1samp_test as pcluster_test
from mne.time_frequency import tfr_multitaper, tfr_morlet
from mne.time_frequency import morlet, fwhm
import tables as tb


In [3]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


## Save in PyTables 

## Preprocess

In [2]:


%matplotlib qt

def read_file(path):
    r = mne.io.read_raw_bdf(path, preload=False)
    
    # Define signal info
    n_time_samps = r.n_times
    time_secs = r.times
    ch_names = r.ch_names
    n_chan = len(ch_names) 
    
    
    channels_to_keep = ["A1", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9", "A10",
            "A11", "A12", "A13", "A14", "A15", "A16", "A17", "A18", "A19", "A20",
            "A21", "A22", "A23", "A24", "A25", "A26", "A27", "A28", "A29", "A30",
            "A31", "A32", "B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B9", "B10",
            "B11", "B12", "B13", "B14", "B15", "B16", "B17", "B18", "B19", "B20",
            "B21", "B22", "B23", "B24", "B25", "B26", "B27", "B28", "B29", "B30", "B31", "B32"]

    channels_to_exclude_str = [ch for ch in r.ch_names if ch not in channels_to_keep + ['Status']]
    
    
    raw = mne.io.read_raw_bdf(path, exclude = channels_to_exclude_str ,  preload=True)
    
    list_chan_name = { 'A3': 'AF3',
     'A7': 'F7', 'A6': 'F5','A5': 'F3', 'A4': 'F1',
     'A8': 'FT7', 'A9': 'FC5','A10': 'FC3', 'A11': 'FC1',
     'A15': 'T7', 'A14': 'C5','A13': 'C3', 'A12': 'C1',
     'A16': 'TP7', 'A17': 'CP5','A18': 'CP3', 'A19': 'CP1', 'A32': 'CPz',
     'A24': 'P9', 'A23': 'P7','A22': 'P5', 'A21': 'P3', 'A20': 'P1','A31': 'Pz',
     'A25': 'PO7', 'A26': 'PO3','A30': 'POz', 
     'A27': 'O1',   'A29': 'Oz', 
     'A28': 'Iz', 
     'B1': 'Fpz','B2': 'Fp2',
     'B5': 'AFz', 'B4': 'AF4','B3': 'AF8',
     'B6': 'Fz', 'B7': 'F2','B8': 'F4', 'B9': 'F6','B10': 'F8',
     'B15': 'FCz', 'B14': 'FC2','B13': 'FC4', 'B12': 'FC6','B11': 'FT8',
     'B16': 'Cz', 'B17': 'C2','B18': 'C4', 'B19': 'C6','B20': 'T8',
     'B24': 'CP2', 'B23': 'CP4','B22': 'CP6', 'B21': 'TP8',
     'B25': 'P2', 'B26': 'P4','B27': 'P6', 'B28': 'P8','B29': 'P10',
     'B31': 'PO4', 'B30': 'PO8',
     'B32': 'O2'
     }

    _ = raw.rename_channels(list_chan_name)
    print(raw.info)
    
    
    
    return raw

    
def highlow_pass(raw):
    # We cut off at one Hertz, because we will be looking at the alpha and beta frequency bands that start
# from the 8-10 Hz. Apply high-pass filter
    raw.filter(l_freq=1, h_freq=None, fir_design='firwin', filter_length='auto',
           l_trans_bandwidth='auto', h_trans_bandwidth='auto', method='fir',
           phase='zero', fir_window='hamming', verbose=True)

# Apply low-pass filter if needed
    raw.filter(l_freq=None, h_freq=40, fir_design='firwin', filter_length='auto',
           l_trans_bandwidth='auto', h_trans_bandwidth='auto', method='fir',
           phase='zero', fir_window='hamming', verbose=True)

    return raw 
    
    
def plotsignal(signal, channels, pick_min, pick_max):
    
    ch_names = signal.ch_names
    
    
    picks = ch_names[pick_min:pick_max]  # Pick the first 20 EEG channels
    signal.plot(n_channels=channels, picks=picks, events = False, title='EEG Data for the Channels')
    
    
   
def rename_events(raw):
    
    # Find events and drop unnecessary
    events_ID = mne.find_events(raw, stim_channel="Status")
    events = mne.pick_events(events_ID, exclude=[3, 5, 40, 42, 62])
    
    
    unsuccessful_event_index = None  # For sanity check
    
    for i, event in enumerate(events):
    
        if event[2] == 10:
            
            next_event_index = i + 1    
            while next_event_index < len(events) and events[next_event_index][2] == 61: 
                events[next_event_index][2] = 610 # Modify the trigger value
                next_event_index += 1   
        
        if event[2] == 20:
            next_event_index = i + 1    
            while next_event_index < len(events) and events[next_event_index][2] == 61: 
                events[next_event_index][2] = 620 # Modify the trigger value
                next_event_index += 1   
        
        
        if event[2] == 30:
            next_event_index = i + 1    
            while next_event_index < len(events) and events[next_event_index][2] == 61: 
                events[next_event_index][2] = 630 # Modify the trigger value
                next_event_index += 1   

        else:
            unsuccessful_event_index = i

    # Check if any events with trigger value 61 are still present
    if any(event[2] == 61 for event in events):
        print("Renaming was unsuccessful :(")
        if unsuccessful_event_index is not None:
            print("Problematic event index:", unsuccessful_event_index)
            print("Problematic event details:", events[unsuccessful_event_index])
   
    # Define epoch parameters
    tmin = -2
    tmax = 3

    
    
    # Rename events with numerical IDs to have corresponding event names
    event_dict = {"10": 10, 
                  "20": 20,
                  "30": 30,
                  "selftap": 41, 
                  "VP610": 610,
                  "VP620": 620, 
                  "VP630": 630}
    
    # Epoch signal
    epochs = mne.Epochs(raw, events, event_id = event_dict, tmin=tmin, tmax=tmax, baseline=None)
    
    del raw

    # Might introduce edge artifacts for each epoch.
    epochs.load_data()
    epochs_rs = epochs.resample(sfreq=256)

    # Print events
    event_ids = epochs_rs.events[:, -1]
    print()
    unique_event_ids = set(event_ids)
    print("Unique Event IDs:", unique_event_ids)
    # Print counts of each event ID
    for event_id in unique_event_ids:
        count = (event_ids == event_id).sum()
        print("Event ID:", event_id, "Count:", count)

    return epochs

def drop_bad_interp(epochs, channels_list):
    
    ep_ds = epochs.copy()

    # Highlight bad channels # we keep the flat one as it will be fixed after averaging 
    _ = ep_ds.info["bads"].extend(channels_list)  

    # Set montage and interpolate bads 
    ep_ds.set_montage('standard_1020')
    ep_interp = ep_ds.interpolate_bads(
    reset_bads=False, method='MNE', verbose=True
    )
    
    return ep_interp
    
    
import types 

def delete_variables_except(exceptions):
    """
    Delete all variables in the global namespace except for the specified exceptions.
    
    Parameters:
    exceptions (list): List of variable names to exclude from deletion.
    """
    # Get a list of all variable names in the global namespace
    all_variables = list(globals().keys())
    
    # Iterate through all variables and delete those not in the exceptions list
    for var_name in all_variables:
        if var_name not in exceptions:
            # Check if the object is a variable (not a module, function, etc.)
            if not isinstance(globals()[var_name], (types.FunctionType, types.ModuleType)):
                del globals()[var_name]
    
   


## Saving files 

In [8]:
def save_epochs(sub,data_dir,epochs,name):
    # saves Epoch
    epochs.save(data_dir + sub + name + '-epo.fif', overwrite=True)
    
    
def save_ica(subject,directionary,ica):
        
        ica.save(directionary + subject+ '-ica.fif', 
        overwrite=True)

## ICA and waveanalysis

In [3]:
def fit_ica(epochs, sub):
    
    # run fast ICA 
    ica = mne.preprocessing.ICA(
    n_components=30, method="fastica", max_iter="auto", random_state=97)
    
    ica.fit(epochs) # fit on epochs
    
    # Visualise ICA components
    montage = mne.channels.make_standard_montage("standard_1005") 
    epochs.set_montage(montage)

    ica.plot_sources(epochs) # plot the raw signal 

    #scalp field topographies 
    ica.plot_components() # plot a Topomap and Chooose ICA compoenents.
    data_dir = 'C:/Users/Mabel Ife/OneDrive - Danmarks Tekniske Universitet/Dokumenter/Master in AI/Final Thesis/Data/Ica_files/'
    
    

    save_ica(sub,data_dir,ica)
    
    return ica
    

def get_ica_sources(epochs, list_of_Ica_components, ica):
    
    
    all_sources = ica.get_sources(epochs)

    
    # Get the sources corresponding to the components
    motor_source = all_sources.pick(list_of_Ica_components)
    
    # separate conditions
    motor_source_event41 = motor_source['selftap']
    motor_source_event610 = motor_source['VP610']
    motor_source_event620 = motor_source['VP620']
    motor_source_event630 = motor_source['VP630']
    
    
    # combine all conditions to one list 
    all_motor_components = [motor_source_event41, motor_source_event610, motor_source_event620, motor_source_event630]
    
    return all_motor_components




def basecorrection(p):
    
    # Input average across frequency 
    
    # Define baseline period
    baseline_tmin = -0.5
    baseline_tmax = 1.5

    time_interval = [baseline_tmin, baseline_tmax]
    
    interval_baseline = [i for i in p.times if i >= time_interval[0] and i <= time_interval[1]]
    baseline = []
    
    # Find the indices corresponding to the time interval
    start_index = np.where(p.times >= time_interval[0])[0][0]
    end_index = np.where(p.times <= time_interval[1])[0][-1]

    
    # Calculate baseline
    for e in range(len(p.data)):
        avg_epoch = np.mean(p.data[e, :, :, start_index:end_index], axis=-1)
        baseline.append(avg_epoch)
    baseline = np.asarray(baseline)

    
    print("Shape of p.data:", p.data.shape)
    print("Shape of baseline:", baseline.shape)
    
    baseline_reshaped = np.expand_dims(baseline, axis=-1) 
    
    print("Shape of baseline reshaped:", baseline_reshaped.shape)
    
    
    # Subtract baseline and normalize
    for e in range(len(p.data)):
        for i in range(len(p.ch_names)):
            p.data[e, i, :] -= baseline_reshaped[e, i]  # Subtract baseline
            p.data[e, i, :] /= baseline_reshaped[e, i]  # Normalize

    
    return p
      
      
def baseCorrection(p):
    
    p = p.copy()
    # Get statistics of data before correction
    print("Before baseline correction:")
    print("Mean:", np.mean(p.data))
    print("Min:", np.min(p.data))
    print("Max:", np.max(p.data))
    
    print("Standard deviation:", np.std(p.data))
    #plt.hist(p.data.ravel(), bins=50)
    #plt.title('Histogram of Data Before Correction')
    #plt.show()

    # Perform baseline correction
    baseline_tmin = -0.5
    baseline_tmax = 1.5
    baseline = (baseline_tmin, baseline_tmax)
    p.data = mne.baseline.rescale(p.data, p.times, baseline, mode='percent', copy=True, verbose=None)

        
    # Get statistics of data after correction
    print("After baseline correction:")
    print("Mean:", np.mean(p.data))
    print("Min:", np.min(p.data))
    print("Max:", np.max(p.data))
    print("Standard deviation:", np.std(p.data))
    #plt.hist(p.data.ravel(), bins=50)
    #plt.title('Histogram of Data After Correction')
    #plt.show()
    return p

def wave_analysis(motor_source):


    # Perform wavelet analysis
    freqs = np.arange(1, 31) # interested in frequencies 1 to 30 hz
    n_cycles = 3 + 0.125 * freqs # Number of cycles   
                                
    p = mne.time_frequency.tfr_morlet(motor_source, freqs, n_cycles, use_fft=False, return_itc=False,

                        decim=1, n_jobs=None, zero_mean=True, average=False, output='power',

                        picks="all")
    
    
    
    # Output p datandarray, shape (n_epochs, n_channels, n_freqs, n_times)
    
    return  p



      

## Get ERD's

In [6]:
def Get_alpha_beta_comps(Basecorrect_epochs):
    
    
    # Define frequency bands
    alpha_band = [8, 12]
    beta_band = [13, 25]

    # Define time window
    tmin = -0.5
    tmax = 1.5
    baseline = (tmin,tmax)

    
    cropped_epochs = Basecorrect_epochs.copy().crop(tmin, tmax)
    #cropped_epochs  = Basecorrect_epochs.apply_baseline(baseline)

    
    p_alpha_left = cropped_epochs.copy().crop(fmin = alpha_band[0], fmax = alpha_band[1]).pick(picks = ['ICA009']).average(dim = 'freqs')
    p_beta_left = cropped_epochs.copy().crop(fmin = beta_band[0], fmax = beta_band[1]).pick(picks = ['ICA009']).average(dim = 'freqs')
    p_alpha_right = cropped_epochs.copy().crop(fmin = alpha_band[0], fmax = alpha_band[1]).pick(picks = ['ICA008']).average(dim = 'freqs')
    p_beta_right = cropped_epochs.copy().crop(fmin = beta_band[0], fmax = beta_band[1]).pick(picks = ['ICA008']).average(dim = 'freqs')

    
    
    print("Shape of epochs data averaged across alpha band for left component:", p_alpha_left.data.shape)
    print("Timepoints for above mentioned data:", len(p_alpha_left.times))
    
    # Compute ERD &  # datandarray, shape (n_channels, n_freqs, n_times)
    erd_alpha_left = p_alpha_left.average(dim = 'epochs')
    erd_beta_left = p_beta_left.average(dim = 'epochs')
    erd_alpha_right = p_alpha_right.average(dim = 'epochs')
    erd_beta_right = p_beta_right.average(dim = 'epochs')
    
    print(" ")
    print("Shape of epochs data averaged across trials for left component:", erd_alpha_left.data.shape)
    
    return erd_alpha_left, erd_alpha_right, erd_beta_left, erd_beta_right
    #return p_alpha_left, p_alpha_right, p_beta_left, p_beta_right
    


In [4]:
# Loads data, filters raw signal, artefact rejection, average
sample_fname = "C:/Users/Mabel Ife/OneDrive - Danmarks Tekniske Universitet/Dokumenter/Master in AI/Final Thesis/Data/subj_07_070223_task.bdf"
raw_filt = highlow_pass(read_file(sample_fname))  # reads file and performs high and low pass filtering
epochs_rs = rename_events(raw_filt)# renames events, create epochs, resample to 256HZ


ep_interp = drop_bad_interp(epochs_rs,['TP8']) # artefact rejection and interpolation of channels

# use the average of all channels as reference
rerefferenced_epochs, ref_data = mne.set_eeg_reference(ep_interp, ref_channels="average", projection=False)


Extracting EDF parameters from C:\Users\Mabel Ife\OneDrive - Danmarks Tekniske Universitet\Dokumenter\Master in AI\Final Thesis\Data\subj_07_070223_task.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\Mabel Ife\OneDrive - Danmarks Tekniske Universitet\Dokumenter\Master in AI\Final Thesis\Data\subj_07_070223_task.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 4517887  =      0.000 ...  2206.000 secs...
<Info | 8 non-empty values
 bads: []
 ch_names: A1, A2, AF3, F1, F3, F5, F7, FT7, FC5, FC3, FC1, C1, C3, C5, T7, ...
 chs: 64 EEG, 1 Stimulus
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 417.0 Hz
 meas_date: 2023-02-07 14:46:06 UTC
 nchan: 65
 projs: []
 sfreq: 2048.0 Hz
 subject_info: 1 item (dict)
>
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pas

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   16.2s finished


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 677 samples (0.331 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   13.0s finished


Trigger channel Status has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
4237 events found on stim channel Status
Event IDs: [ 3  5 10 20 30 40 41 42 61 62]
Not setting metadata
1707 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1707 events and 10241 original time points ...
0 bad epochs dropped

Unique Event IDs: {610, 41, 10, 620, 20, 630, 30}
Event ID: 610 Count: 450
Event ID: 41 Count: 540
Event ID: 10 Count: 30
Event ID: 620 Count: 301
Event ID: 20 Count: 30
Event ID: 630 Count: 326
Event ID: 30 Count: 30


In [5]:




epochs_trial41 = rerefferenced_epochs['selftap']
epochs_trial610 = rerefferenced_epochs['VP610']
epochs_trial620 = rerefferenced_epochs['VP620']
epochs_trial630 = rerefferenced_epochs['VP630']


p_id = 'sub-007'
data_dir = 'C:/Users/Mabel Ife/OneDrive - Danmarks Tekniske Universitet/Dokumenter/Master in AI/Final Thesis/Data/epochs/'

save_epochs(p_id,data_dir,epochs_trial41, '-AVtrial41')
save_epochs(p_id,data_dir,epochs_trial610, '-AVtrial610')
save_epochs(p_id,data_dir,epochs_trial620, '-AVtrial620')
save_epochs(p_id,data_dir,epochs_trial630, '-AVtrial630')


Setting channel interpolation method to {'eeg': 'MNE'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.6 mm
    Computing dot products for 63 EEG channels...
    Computing cross products for 63 → 64 EEG channels...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


    Preparing the mapping matrix...
    Truncating at 63/63 components and regularizing with α=1.0e-01
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


NameError: name 'save_epochs' is not defined

In [30]:


current_directory = "C:/Users/Mabel Ife/OneDrive - Danmarks Tekniske Universitet/Dokumenter/Thesis Code"
unpickled_wave_list[0].save(current_directory +  '-selftap_power.fif', overwrite=True)

RuntimeError: For HDF5-based I/O to work, the h5io module is needed, but it could not be imported.
              use the following installation method appropriate for your environment:
              'pip install h5io'
              'conda install -c conda-forge h5io'

For the subject 7, we found the two components 8 and 9. 

## ICA and Wavelet analysis 

In [6]:
# Free-up memory

delete_variables_except(['rerefferenced_epochs'])

In [9]:
# fit ICA and look for motor components
p_id = 'sub-007'
ICA = fit_ica(rerefferenced_epochs,p_id)

# Pick components (comment out when needed)
ICA.plot_properties(ep_avg_ref, picks=[8,9], log_scale=False)


# get sources for components
all_conditions = get_ica_sources(rerefferenced_epochs,['ICA008','ICA009'],ICA)

# Run wavelet and basecorrection 

#baseCorr_power = [] # saves baselinecorrected epochs 




Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 30 components
Fitting ICA took 279.8s.
Not setting metadata
1707 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Writing ICA solution to C:\Users\Mabel Ife\OneDrive - Danmarks Tekniske Universitet\Dokumenter\Master in AI\Final Thesis\Data\Ica_files\sub-007-ica.fif...


In [ ]:
# Run wavelet and basecorrection 


after_wave  = []
for motor_source_condition in all_conditions: # loop through each condition
    p_df = wave_analysis(motor_source_condition)
    after_wave.append(p_df)
    
    


In [ ]:
file_path = 'Power.pickle'

# Open the file in binary mode
with open(file_path, 'wb') as file:
    # Serialize and write the variable to the file
    pickle.dump(after_wave, file)

In [6]:
import numpy as np 
def baseCorrection_ERDS(epochs_power):
    
    # First we average across frequency in alpha and beta band 
    av_freq_alpha = epochs_power.copy().crop(fmin=8, fmax=12).average(dim='freqs')
    av_freq_beta = epochs_power.copy().crop(fmin=15, fmax=25).average(dim='freqs')
    
    
                                               
    # average across trials
    av_epoch_alpha = av_freq_alpha.copy().crop(tmin=-0.5, tmax=1.5).average()
    av_epoch_beta = av_freq_beta.copy().crop(tmin=-0.5, tmax=1.5).average()
    
      # Get statistics of data before correction
    print("Statistics Before baseline correction for alpha:")
    print("Mean:", np.mean(av_epoch_alpha.data))
    print("Min:", np.min(av_epoch_alpha.data))
    print("Max:", np.max(av_epoch_alpha.data))
    
    print("Standard deviation:", np.std(av_epoch_alpha.data))
    
    
    # Baseline correct ERD's
    baseline = (-0.5,1.5)
    erds_alpha = av_epoch_alpha.copy().apply_baseline(baseline, mode='percent', verbose=None)
    erds_beta = av_epoch_beta.copy().apply_baseline(baseline, mode='percent', verbose=None)
    
    
    print("Shape for ERDs alpha:", erds_alpha.data.shape)
    print("Shape for ERDs Beta:", erds_beta.data.shape)
    
      # Get statistics of data before correction
    print(" ")
    print("Statistics after baseline correction for alpha:")
    print("Mean:", np.mean(erds_alpha.data))
    print("Min:", np.min(erds_alpha.data))
    print("Max:", np.max(erds_alpha.data))
    
    print("Standard deviation:", np.std(erds_alpha.data))
    
    
    ERD_pair = (erds_alpha,erds_beta)
    
    
    return ERD_pair
    
def get_Erdslist(powerlist):
    
    erds_list = []
    
    for power in powerlist:
        erdpair = baseCorrection_ERDS(power)
        erds_list.append(erdpair)
    
    print("")    
    print("Statistics after organising erds en tuples for alpha 41 trial:")
    print("Mean:", np.mean(erds_list[0][0].data))
    print("Min:", np.min(erds_list[0][0].data))
    print("Max:", np.max(erds_list[0][0].data))
    
    print("Standard deviation:", np.std(erds_list[0][0].data))   
        
    return erds_list
        
    

In [13]:
import numpy as np 
def baseCorrection_ERDS(epochs_power):
    
    # First we average across frequency in alpha and beta band 
    av_freq_alpha = epochs_power.copy().crop(fmin=8, fmax=12).average(dim='freqs')
    av_freq_beta = epochs_power.copy().crop(fmin=20, fmax=30).average(dim='freqs')
    
    

    # Baseline correct ERD's
    baseline = (-0.5,1.5)
    erds_alpha = av_freq_alpha.copy().apply_baseline(baseline, mode='percent', verbose=None)
    erds_beta =  av_freq_beta.copy().apply_baseline(baseline, mode='percent', verbose=None)
    
    
                                                   
    # average across trials
    av_epoch_alpha =  erds_alpha.copy().crop(tmin=-0.5, tmax=1.5).average()
    av_epoch_beta = erds_beta.copy().crop(tmin=-0.5, tmax=1.5).average()
    
  
    
    ERD_pair = (av_epoch_alpha,av_epoch_beta)
    
    
    return ERD_pair
    
def get_Erdslist(powerlist):
    
    erds_list = []
    
    for power in powerlist:
        erdpair = baseCorrection_ERDS_new(power)
        erds_list.append(erdpair)
    
    print("")    
    print("Statistics after organising erds en tuples for alpha 41 trial:")
    print("Mean:", np.mean(erds_list[0][0].data))
    print("Min:", np.min(erds_list[0][0].data))
    print("Max:", np.max(erds_list[0][0].data))
    
    print("Standard deviation:", np.std(erds_list[0][0].data))   
        
    return erds_list
        
    

In [14]:
# Get ERDS 

ERDS_all_new = get_Erdslist_new(unpickled_wave_list)

Applying baseline correction (mode: percent)
Applying baseline correction (mode: percent)
Applying baseline correction (mode: percent)
Applying baseline correction (mode: percent)
Applying baseline correction (mode: percent)
Applying baseline correction (mode: percent)
Applying baseline correction (mode: percent)
Applying baseline correction (mode: percent)

Statistics after organising erds en tuples for alpha 41 trial:
Mean: -3.289549702593056e-17
Min: -0.0984571775090432
Max: 0.21222213241662677
Standard deviation: 0.05578307572336199


In [23]:
df = unpickled_wave_list[0].to_data_frame()

In [24]:
# saving the dataframe
df.to_csv('selftap_power.csv')

# Plot that works


In [ ]:
def plot1ERDS(Erdslist):
    # Unpack the ERDs data from the list
    alpha = [data[0] for data in Erdslist]
    beta = [data[1] for data in Erdslist]
    
    
    t1 = np.linspace(-0.5, 1.5, 513)  # time-axis
    
    
    fig, ax = plt.subplots(figsize=(20, 10))
    
    

    # plot1 ICA LEft ALpha 
    ax.set_xlabel('Time Instances')
    ax.set_ylabel('Volt')

    ax.plot(t1, alpha[0].data[1,0,:], color='blue', label='self Tap')
    ax.plot(t1, alpha[1].data[1,0,:], color='red', label='Non-adaptive')
    ax.plot(t1, alpha[2].data[1,0,:], color='green', label='Moderately adaptive')
    ax.plot(t1, alpha[3].data[1,0,:], color='pink', label='Overly adaptive')

    # Adding vertical lines at specified time points
    ax.axvline(x=0, color='black', linestyle='--', linewidth=1)  # at time point 0
    ax.axvline(x=1, color='black', linestyle='--', linewidth=1)  # at time point 1
    ax.axvline(x=1.5, color='black', linestyle='--', linewidth=1)  # at time point 1.5

    legend = ax.legend(loc='upper right', shadow=True, fontsize='medium')
    plt.title('ERD left ICA (ICA009) for Alpha-frequencies')
    
    plt.show()
    
plot1ERDS(ERDS_all)

In [35]:
y = ERDS_power[0][0].data[0,0,:]

In [36]:
import numpy as np
import plotly.graph_objs as go

# Sample data (replace this with your actual data)
t1 = np.linspace(-0.5, 1.5, 513)  # 512 samples with 256 Hz sampling rate


# Creating traces
trace = go.Scatter(
    x=t1,
    y=y,
    mode='lines',
    line=dict(color='pink'),
    name='self Tap with Basecorrection'
)

# Adding vertical lines at specified time points
vertical_lines = [
    dict(type="line", x0=0, y0=min(y), x1=0, y1=max(y), line=dict(color="black", width=1, dash="dash")),
    dict(type="line", x0=1, y0=min(y), x1=1, y1=max(y), line=dict(color="black", width=1, dash="dash")),
    dict(type="line", x0=1.5, y0=min(y), x1=1.5, y1=max(y), line=dict(color="black", width=1, dash="dash"))
]

layout = go.Layout(
    title='Alpha [ICA008] Selftap',
    xaxis=dict(title='Time Instances'),
    yaxis=dict(title='Volt'),
    shapes=vertical_lines
)

# Creating the figure
fig = go.Figure(data=[trace], layout=layout)

# Show plot
fig.show()


In [29]:
import numpy as np
import plotly.graph_objs as go

# Sample data (replace this with your actual data)
t1 = np.linspace(-0.5, 1.5, 513)  # 512 samples with 256 Hz sampling rate


# Creating traces
trace = go.Scatter(
    x=t1,
    y=y,
    mode='lines',
    line=dict(color='pink'),
    name='self Tap with Basecorrection'
)

# Adding vertical lines at specified time points
vertical_lines = [
    dict(type="line", x0=0, y0=min(y), x1=0, y1=max(y), line=dict(color="black", width=1, dash="dash")),
    dict(type="line", x0=1, y0=min(y), x1=1, y1=max(y), line=dict(color="black", width=1, dash="dash")),
    dict(type="line", x0=1.5, y0=min(y), x1=1.5, y1=max(y), line=dict(color="black", width=1, dash="dash"))
]

layout = go.Layout(
    title='Alpha [ICA008] Selftap',
    xaxis=dict(title='Time Instances'),
    yaxis=dict(title='Volt'),
    shapes=vertical_lines
)

# Creating the figure
fig = go.Figure(data=[trace], layout=layout)

# Show plot
fig.show()


In [10]:
from bokeh.plotting import figure, show
from bokeh.models import Legend
import numpy as np

# Sample data (replace this with your actual data)
t1 = np.linspace(-0.5, 1.5, 513)  # 512 samples with 256 Hz sampling rate
y = ERDS_all[0][0].data[0,0,:]  # Replace this with your actual data

# Create Bokeh figure
p = figure(title='Alpha [ICA008] Selftap', x_axis_label='Time Instances', y_axis_label='Volt')

# Plot data
line = p.line(t1, y, color='pink', legend_label='self Tap with Basecorrection')

# Add vertical lines
vline1 = p.line([0, 0], [min(y), max(y)], color='black', line_dash='dashed', legend_label='Vertical Lines')
vline2 = p.line([1, 1], [min(y), max(y)], color='black', line_dash='dashed')
vline3 = p.line([1.5, 1.5], [min(y), max(y)], color='black', line_dash='dashed')

# Add legend
legend = Legend(items=[('Line', [line]), ('Vertical Lines', [vline1, vline2, vline3])], location='top_right')
p.add_layout(legend)

# Show the plot
show(p)



NameError: name 'ERDS_all' is not defined

In [ ]:
import numpy as np
import plotly.graph_objs as go

# Sample data (replace this with your actual data)
t1 = np.linspace(-0.5, 1.5, 513)  # 512 samples with 256 Hz sampling rate
erd_data = ERDS_all_new[0][0].data[0,0,:]  # Sample random data, replace this with your erd41.data

# Creating traces
trace = go.Scatter(
    x=t1,
    y=erd_data,
    mode='lines',
    line=dict(color='pink'),
    name='self Tap with Basecorrection'
)

# Adding vertical lines at specified time points
vertical_lines = [
    dict(type="line", x0=0, y0=min(erd_data), x1=0, y1=max(erd_data), line=dict(color="black", width=1, dash="dash")),
    dict(type="line", x0=1, y0=min(erd_data), x1=1, y1=max(erd_data), line=dict(color="black", width=1, dash="dash")),
    dict(type="line", x0=1.5, y0=min(erd_data), x1=1.5, y1=max(erd_data), line=dict(color="black", width=1, dash="dash"))
]

layout = go.Layout(
    title='Alpha [ICA008] Selftap',
    xaxis=dict(title='Time Instances'),
    yaxis=dict(title='Volt'),
    shapes=vertical_lines
)

# Creating the figure
fig = go.Figure(data=[trace], layout=layout)

# Show plot
fig.show()


In [19]:
import plotly.graph_objs as go

def plotERDS(Erdslist):
    # Unpack the ERDs data from the list
    alpha = [data[0] for data in Erdslist]
    beta = [data[1] for data in Erdslist]
    
    t1 = np.linspace(-0.5, 1.5, 513)  # time-axis

    # Plot 1: ICA Left Alpha
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x=t1, y=alpha[0].data[1,0,:], mode='lines', name='self Tap', line=dict(color='blue')))
    fig1.add_trace(go.Scatter(x=t1, y=alpha[1].data[1,0,:], mode='lines', name='Non-adaptive', line=dict(color='red')))
    fig1.add_trace(go.Scatter(x=t1, y=alpha[2].data[1,0,:], mode='lines', name='Moderately adaptive', line=dict(color='green')))
    fig1.add_trace(go.Scatter(x=t1, y=alpha[3].data[1,0,:], mode='lines', name='Overly adaptive', line=dict(color='pink')))
    fig1.update_layout(title='ERD left ICA (ICA009) for Alpha-frequencies',
                       xaxis_title='Time Instances', yaxis_title='Volt')
    fig1.add_vline(x=0, line_dash="dash", line_color="black")
    fig1.add_vline(x=0.5, line_dash="dash", line_color="black")
    fig1.add_vline(x=1, line_dash="dash", line_color="black")
    fig1.show()

    # Plot 2: ICA Right Alpha
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=t1, y=alpha[0].data[0,0,:], mode='lines', name='self Tap', line=dict(color='blue')))
    fig2.add_trace(go.Scatter(x=t1, y=alpha[1].data[0,0,:], mode='lines', name='Non-adaptive', line=dict(color='red')))
    fig2.add_trace(go.Scatter(x=t1, y=alpha[2].data[0,0,:], mode='lines', name='Moderately adaptive', line=dict(color='green')))
    fig2.add_trace(go.Scatter(x=t1, y=alpha[3].data[0,0,:], mode='lines', name='Overly adaptive', line=dict(color='orange')))
    fig2.update_layout(title='ERD right ICA (ICA008) for Alpha-frequencies',
                       xaxis_title='Time Instances', yaxis_title='Volt')
    fig2.add_vline(x=0, line_dash="dash", line_color="black")
    fig2.add_vline(x=0.5, line_dash="dash", line_color="black")
    fig2.add_vline(x=1, line_dash="dash", line_color="black")
    fig2.show()

    # Plot 3: ICA Left Beta
    fig3 = go.Figure()
    fig3.add_trace(go.Scatter(x=t1, y=beta[0].data[1,0,:], mode='lines', name='self Tap', line=dict(color='blue')))
    fig3.add_trace(go.Scatter(x=t1, y=beta[1].data[1,0,:], mode='lines', name='Non-adaptive', line=dict(color='red')))
    fig3.add_trace(go.Scatter(x=t1, y=beta[2].data[1,0,:], mode='lines', name='Moderately adaptive', line=dict(color='green')))
    fig3.add_trace(go.Scatter(x=t1, y=beta[3].data[1,0,:], mode='lines', name='Overly adaptive', line=dict(color='pink')))
    fig3.update_layout(title='ERD left ICA (ICA009) for Beta-frequencies',
                       xaxis_title='Time Instances', yaxis_title='Volt')
    fig3.add_vline(x=0, line_dash="dash", line_color="black")
    fig3.add_vline(x=0.5, line_dash="dash", line_color="black")
    fig3.add_vline(x=1, line_dash="dash", line_color="black")
    fig3.show()

    # Plot 4: ICA Right Beta
    fig4 = go.Figure()
    fig4.add_trace(go.Scatter(x=t1, y=beta[0].data[0,0,:], mode='lines', name='self Tap', line=dict(color='blue')))
    fig4.add_trace(go.Scatter(x=t1, y=beta[1].data[0,0,:], mode='lines', name='Non-adaptive', line=dict(color='red')))
    fig4.add_trace(go.Scatter(x=t1, y=beta[2].data[0,0,:], mode='lines', name='Moderately adaptive', line=dict(color='green')))
    fig4.add_trace(go.Scatter(x=t1, y=beta[3].data[0,0,:], mode='lines', name='Overly adaptive', line=dict(color='orange')))
    fig4.update_layout(title='ERD right ICA (ICA008) for Beta-frequencies',
                       xaxis_title='Time Instances', yaxis_title='Volt')
    fig4.add_vline(x=0, line_dash="dash", line_color="black")
    fig4.add_vline(x=0.5, line_dash="dash", line_color="black")
    fig4.add_vline(x=1, line_dash="dash", line_color="black")
    fig4.show()

plotERDS(ERDS_all_new)


In [20]:
from bokeh.plotting import figure, gridplot, show
import numpy as np

# Sample data (replace this with your actual data)
t1 = np.linspace(-0.5, 1.5, 513)  # 512 samples with 256 Hz sampling rate
y = ERDS_all_new[0][1].data[0,0,:]
y1 = ERDS_all_new[0][1].data[0,0,:]
y2 = ERDS_all_new[0][1].data[0,0,:]
y3 = ERDS_all_new[0][1].data[0,0,:]

# Create Bokeh figures
p1 = figure(title='Beta [ICA008] Selftap 1', x_axis_label='Time Instances', y_axis_label='Volt')
p1.line(t1, y, color='pink')
p1.line([0, 0], [min(y), max(y)], color='black', line_dash='dashed')
p1.line([0.5, 0.5], [min(y), max(y)], color='black', line_dash='dashed')
p1.line([1, 1], [min(y), max(y)], color='black', line_dash='dashed')
show(p1)

In [ ]:
from bokeh.plotting import figure, gridplot, show
import numpy as np

# Sample data (replace this with your actual data)
t1 = np.linspace(-0.5, 1.5, 513)  # 512 samples with 256 Hz sampling rate
y = ERDS_all[0][0].data[0,0,:]
y1 = ERDS_all[0][0].data[0,0,:]
y2 = ERDS_all[0][0].data[0,0,:]
y3 = ERDS_all[0][0].data[0,0,:]

# Create Bokeh figures
p1 = figure(title='Alpha [ICA008] Selftap 1', x_axis_label='Time Instances', y_axis_label='Volt')
p1.line(t1, y, color='pink')
p1.line([0, 0], [min(y), max(y)], color='black', line_dash='dashed')
p1.line([0.5, 0.5], [min(y), max(y)], color='black', line_dash='dashed')
p1.line([1, 1], [min(y), max(y)], color='black', line_dash='dashed')

p2 = figure(title='Alpha [ICA008] Selftap 2', x_axis_label='Time Instances', y_axis_label='Volt')
p2.line(t1, y, color='pink')
p2.line([0, 0], [min(y), max(y)], color='black', line_dash='dashed')
p2.line([0.5, 0.5], [min(y), max(y)], color='black', line_dash='dashed')
p2.line([1, 1], [min(y), max(y)], color='black', line_dash='dashed')

p3 = figure(title='Alpha [ICA008] Selftap 3', x_axis_label='Time Instances', y_axis_label='Volt')
p3.line(t1, y, color='pink')
p3.line([0, 0], [min(y), max(y)], color='black', line_dash='dashed')
p3.line([0.5, 0.5], [min(y), max(y)], color='black', line_dash='dashed')
p3.line([1, 1], [min(y), max(y)], color='black', line_dash='dashed')

p4 = figure(title='Alpha [ICA008] Selftap 4', x_axis_label='Time Instances', y_axis_label='Volt')
p4.line(t1, y, color='pink')
p4.line([0, 0], [min(y), max(y)], color='black', line_dash='dashed')
p4.line([0.5, 0.5], [min(y), max(y)], color='black', line_dash='dashed')
p4.line([1, 1], [min(y), max(y)], color='black', line_dash='dashed')

# Create a grid of plots
grid = gridplot([[p1, p2], [p3, p4]])

# Show the grid of plots
show(grid)


In [ ]:
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
import numpy as np

def plotERDS_bokeh(Erdslist):
    # Unpack the ERDs data from the list
    alpha = [data[0] for data in Erdslist]
    beta = [data[1] for data in Erdslist]
    
    t1 = np.linspace(-0.5, 1.5, 513)  # time-axis

    # Plot 1: ICA Left Alpha
    p1 = figure(title='ERD left ICA (ICA009) for Alpha-frequencies', x_axis_label='Time Instances', y_axis_label='Volt')
    p1.line(t1, alpha[0].data[1,0,:], legend_label='self Tap', color='blue')
    p1.line(t1, alpha[1].data[1,0,:], legend_label='Non-adaptive', color='red')
    p1.line(t1, alpha[2].data[1,0,:], legend_label='Moderately adaptive', color='green')
    p1.line(t1, alpha[3].data[1,0,:], legend_label='Overly adaptive', color='pink')
    p1.legend.location = 'top_right'
    p1.line([0, 0], [min(y), max(y)], line_dash="dashed", line_color="black")
    p1.line([0.5, 0.5], [min(y), max(y)], line_dash="dashed", line_color="black")
    p1.line([1, 1], [min(y), max(y)], line_dash="dashed", line_color="black")

    # Plot 2: ICA Right Alpha
    p2 = figure(title='ERD right ICA (ICA008) for Alpha-frequencies', x_axis_label='Time Instances', y_axis_label='Volt')
    p2.line(t1, alpha[0].data[0,0,:], legend_label='self Tap', color='blue')
    p2.line(t1, alpha[1].data[0,0,:], legend_label='Non-adaptive', color='red')
    p2.line(t1, alpha[2].data[0,0,:], legend_label='Moderately adaptive', color='green')
    p2.line(t1, alpha[3].data[0,0,:], legend_label='Overly adaptive', color='orange')
    p2.legend.location = 'top_right'
    p2.line([0, 0], [min(y), max(y)], line_dash="dashed", line_color="black")
    p2.line([0.5, 0.5], [min(y), max(y)], line_dash="dashed", line_color="black")
    p2.line([1, 1], [min(y), max(y)], line_dash="dashed", line_color="black")

    # Plot 3: ICA Left Beta
    p3 = figure(title='ERD left ICA (ICA009) for Beta-frequencies', x_axis_label='Time Instances', y_axis_label='Volt')
    p3.line(t1, beta[0].data[1,0,:], legend_label='self Tap', color='blue')
    p3.line(t1, beta[1].data[1,0,:], legend_label='Non-adaptive', color='red')
    p3.line(t1, beta[2].data[1,0,:], legend_label='Moderately adaptive', color='green')
    p3.line(t1, beta[3].data[1,0,:], legend_label='Overly adaptive', color='pink')
    p3.legend.location = 'top_right'
    p3.line([0, 0], [min(y), max(y)], line_dash="dashed", line_color="black")
    p3.line([0.5, 0.5], [min(y), max(y)], line_dash="dashed", line_color="black")
    p3.line([1, 1], [min(y), max(y)], line_dash="dashed", line_color="black")

    # Plot 4: ICA Right Beta
    p4 = figure(title='ERD right ICA (ICA008) for Beta-frequencies', x_axis_label='Time Instances', y_axis_label='Volt')
    p4.line(t1, beta[0].data[0,0,:], legend_label='self Tap', color='blue')
    p4.line(t1, beta[1].data[0,0,:], legend_label='Non-adaptive', color='red')
    p4.line(t1, beta[2].data[0,0,:], legend_label='Moderately adaptive', color='green')
    p4.line(t1, beta[3].data[0,0,:], legend_label='Overly adaptive', color='orange')
    p4.legend.location = 'top_right'
    p4.line([0, 0], [min(y), max(y)], line_dash="dashed", line_color="black")
    p4.line([0.5, 0.5], [min(y), max(y)], line_dash="dashed", line_color="black")
    p4.line([1, 1], [min(y), max(y)], line_dash="dashed", line_color="black")

    # Arrange the plots in a grid
    grid = gridplot([[p1, p2], [p3, p4]])

    # Show the grid of plots
    show(grid)

plotERDS_bokeh(ERDS_all_new)


## plot ERD's Old plots or failed plots


In [ ]:
%matplotlib qt

def plotERDS(Erdslist):
    # Unpack the ERDs data from the list
    alpha = [data[0] for data in Erdslist]
    beta = [data[1] for data in Erdslist]
    
    
    t1 = np.linspace(-0.5, 1.5, 513)  # time-axis
    
    
    fig, ax = plt.subplots(figsize=(20, 10))
    
    

    # plot1 ICA LEft ALpha 
    ax.set_xlabel('Time Instances')
    ax.set_ylabel('Volt')

    ax.plot(t1, alpha[0].data[1,0,:], color='blue', label='self Tap')
    ax.plot(t1, alpha[1].data[1,0,:], color='red', label='Non-adaptive')
    ax.plot(t1, alpha[2].data[1,0,:], color='green', label='Moderately adaptive')
    ax.plot(t1, alpha[3].data[1,0,:], color='pink', label='Overly adaptive')

    # Adding vertical lines at specified time points
    ax.axvline(x=0, color='black', linestyle='--', linewidth=1)  # at time point 0
    ax.axvline(x=0.5, color='black', linestyle='--', linewidth=1)  # at time point 1
    ax.axvline(x=1, color='black', linestyle='--', linewidth=1)  # at time point 1.5

    legend = ax.legend(loc='upper right', shadow=True, fontsize='medium')
    plt.title('ERD left ICA (ICA009) for Alpha-frequencies')
    
    plt.show()
        
    
    # plot2 to ICA right Alpha
    fig, ax1 = plt.subplots(figsize=(20, 10))
    
    ax1.set_xlabel('Time Instances')
    ax1.set_ylabel('Volt')

    ax1.plot(t1, alpha[0].data[0,0,:], color='blue', label='self Tap')
    ax1.plot(t1, alpha[1].data[0,0,:], color='red', label='Non-adaptive')
    ax1.plot(t1, alpha[2].data[0,0,:], color='green', label='Moderately adaptive')
    ax1.plot(t1, alpha[3].data[0,0,:], color='orange', label='Overly adaptive')

    # Adding vertical lines at specified time points
    ax1.axvline(x=0, color='black', linestyle='--', linewidth=1)  # at time point 0
    ax1.axvline(x=0.5, color='black', linestyle='--', linewidth=1)  # at time point 1
    ax1.axvline(x=1, color='black', linestyle='--', linewidth=1)  # at time point 1.5

    legend = ax1.legend(loc='upper right', shadow=True, fontsize='medium')
    plt.title('ERD right ICA (ICA008) for Alpha-frequencies')
    
      
    plt.show()
       
     # plot3 ICA LEft Beta    
       
    fig, ax2 = plt.subplots(figsize=(20, 10))
    

    ax2.set_xlabel('Time Instances')
    ax2.set_ylabel('Volt')

    ax2.plot(t1, beta[0].data[1,0,:], color='blue', label='self Tap')
    ax2.plot(t1, beta[1].data[1,0,:], color='red', label='Non-adaptive')
    ax2.plot(t1, beta[2].data[1,0,:], color='green', label='Moderately adaptive')
    ax2.plot(t1, beta[3].data[1,0,:], color='pink', label='Overly adaptive')

    # Adding vertical lines at specified time points
    ax2.axvline(x=0, color='black', linestyle='--', linewidth=1)  # at time point 0
    ax2.axvline(x=0.5, color='black', linestyle='--', linewidth=1)  # at time point 1
    ax2.axvline(x=1, color='black', linestyle='--', linewidth=1)  # at time point 1.5

    legend = ax2.legend(loc='upper right', shadow=True, fontsize='medium')
    plt.title('ERD left ICA (ICA009) for beta-frequencies')
    
    plt.show()
        
    # plot 4
    fig, ax3 = plt.subplots(figsize=(20, 10))

    ax3.set_xlabel('Time Instances')
    ax3.set_ylabel('Volt')

    ax3.plot(t1, alpha[0].data[0,0,:], color='blue', label='self Tap')
    ax3.plot(t1, alpha[1].data[0,0,:], color='red', label='Non-adaptive')
    ax3.plot(t1, alpha[2].data[0,0,:], color='green', label='Moderately adaptive')
    ax3.plot(t1, alpha[3].data[0,0,:], color='orange', label='Overly adaptive')

    # Adding vertical lines at specified time points
    ax3.axvline(x=0, color='black', linestyle='--', linewidth=1)  # at time point 0
    ax3.axvline(x=0.5, color='black', linestyle='--', linewidth=1)  # at time point 1
    ax3.axvline(x=1, color='black', linestyle='--', linewidth=1)  # at time point 1.5

    legend = ax3.legend(loc='upper right', shadow=True, fontsize='medium')
    plt.title('ERD right ICA (ICA008) for Beta-frequencies')
     
    
    plt.show()


plotERDS(ERDS_all)

    
    
    

In [ ]:

    # Get statistics of data before correction
print("Without function the selftap condition after baseline:")
print("Mean:", np.mean(erd41.data))
print("Min:", np.min(p.data))
print("Max:", np.max(p.data))
    
print("Standard deviation:", np.std(p.data))
    #plt.hist(p.data.ravel(), bins=50)
    #plt.title('Histogram of Data Before Correction')
    #plt.show()

    # Get statistics of data after correction
print("After baseline correction:")
print("Mean:", np.mean(p.data))
print("Min:", np.min(p.data))
print("Max:", np.max(p.data))
print("Standard deviation:", np.std(p.data))
    #plt.hist(p.data.ravel(), bins=50)
    #plt.title('Histogram of Data After Correction')
    #plt.show()
    return p

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mne import create_info, EpochsArray
from mne.time_frequency import EpochsTFR

del epochs_tfr

# Define your basecorrection function
# Define your baseline correction function using MNE's baseline.rescale
def basecorrection(p):
    # Get statistics of data before correction
    print("Before baseline correction:")
    print("Mean:", np.mean(p.data))
    print("Min:", np.min(p.data))
    print("Max:", np.max(p.data))
    
    print("Standard deviation:", np.std(p.data))
    #plt.hist(p.data.ravel(), bins=50)
    #plt.title('Histogram of Data Before Correction')
    #plt.show()

    # Perform baseline correction
    baseline_tmin = -0.5
    baseline_tmax = 1.5
    baseline = (baseline_tmin, baseline_tmax)
    p.data = mne.baseline.rescale(p.data, p.times, baseline, mode='percent', copy=True, verbose=None)

    # Get statistics of data after correction
    print("After baseline correction:")
    print("Mean:", np.mean(p.data))
    print("Min:", np.min(p.data))
    print("Max:", np.max(p.data))
    print("Standard deviation:", np.std(p.data))
    #plt.hist(p.data.ravel(), bins=50)
    #plt.title('Histogram of Data After Correction')
    #plt.show()
    return p

# Generate synthetic EpochsTFR data
n_epochs = 20
n_channels = 5
n_freqs = 10
n_times = 100

# Create dummy info
sfreq = 256 # Hz
info = create_info(n_channels, sfreq)

# Create dummy epochs array
data = np.random.randn(n_epochs, n_channels, n_times)
epochs = EpochsArray(data, info)

# Convert EpochsArray to EpochsTFR
freqs = np.linspace(6, 30, n_freqs)  # Hz
times = np.arange(n_times) / sfreq
data_tfr = np.random.randn(n_epochs, n_channels, n_freqs, n_times)
epochs_tfr = EpochsTFR(info, data_tfr, times, freqs)

# Apply baseline correction
epochs_tfr_corrected = basecorrection(epochs_tfr)

# Plotting example
epoch_to_plot = 0
channel_to_plot = 0
freq_to_plot = 3  # Index of frequency to plot
plt.figure(figsize=(10, 5))
plt.plot(epochs_tfr.times, epochs_tfr.data[epoch_to_plot, channel_to_plot, freq_to_plot, :], label='Original', linewidth= 3)
plt.plot(epochs_tfr_corrected.times, epochs_tfr_corrected.data[epoch_to_plot, channel_to_plot, freq_to_plot, :], label='Corrected')
plt.xlabel('Time (s)')
plt.ylabel('Power')
plt.title(f'Epoch {epoch_to_plot + 1}, Channel {channel_to_plot + 1}, Frequency {freqs[freq_to_plot]} Hz')
plt.legend()
plt.show()


Not setting metadata
20 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
Before baseline correction:
Mean: 0.0041856305893090105
Min: -4.624396824012296
Max: 4.422527213160561
Standard deviation: 0.9987201832067439
Applying baseline correction (mode: percent)


After baseline correction:
Mean: 1.8467005702405003e-16
Min: -29725.70552012432
Max: 24387.423098578718
Standard deviation: 352.2983036922334
